In [ ]:

[root@bosprod01 Downloads]# ifconfig -a

inet 192.168.153.139  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod02 Downloads]# ifconfig -a

inet 192.168.153.140  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod03 ~]# ifconfig -a

inet 192.168.153.141  netmask 255.255.255.0  broadcast 192.168.153.255


Add host ips in /etc/hosts file and stop firewalld in all servers 
---------------------------------------------------------------------------

[root@bosprod02 ~]# cat /etc/hosts
192.168.153.139 bosprod01
192.168.153.140 bosprod02
192.168.153.141 bosprod03
[root@bosprod02 ~]# 

systemctl stop firewalld

In [ ]:
Configuring an instance for group replication
==============================================================


[root@bosprod01 ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

disabled_storage_engines="MyISAM,BLACKHOLE,FEDERATED,ARCHIVE,MEMORY"

server_id=1
gtid_mode=ON
enforce_gtid_consistency=ON

plugin_load_add='group_replication.so'
group_replication_group_name="5842bb1d-28ef-11ef-9a91-000c293ea0bc"
group_replication_start_on_boot=off
group_replication_local_address= "bosprod01:33061"
group_replication_group_seeds= "bosprod01:33061,bosprod02:33061,bosprod03:33061"
group_replication_bootstrap_group=off
[root@bosprod01 ~]# 


[root@bosprod01 ~]# mysqld --validate-config
[root@bosprod01 ~]# 
[root@bosprod01 ~]# systemctl restart mysqld
[root@bosprod01 ~]# 


In [ ]:
User Credentials For Distributed Recovery
============================================================

mysql> set sql_log_bin = 0;
Query OK, 0 rows affected (0.00 sec)

mysql> create user replicator identified by 'Repl@2024';
Query OK, 0 rows affected (0.19 sec)

mysql> grant replication slave on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> grant connection_admin on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> grant backup_admin on *.* to replicator;
Query OK, 0 rows affected (0.00 sec)

mysql> grant group_replication_stream on *.* to replicator;
Query OK, 0 rows affected (0.00 sec)

mysql> flush privileges;
Query OK, 0 rows affected (0.01 sec)

mysql> set sql_log_bin = 1;
Query OK, 0 rows affected (0.00 sec)

In [ ]:
Launching Group Replication
=============================================


mysql> CHANGE REPLICATION SOURCE TO SOURCE_USER='replicator' ,
    -> SOURCE_PASSWORD='Repl@2024'
    -> FOR CHANNEL 'group_replication_recovery';
Query OK, 0 rows affected, 2 warnings (0.03 sec)


mysql> show plugins;
+---------------------------------+----------+--------------------+----------------------+---------+
| Name                            | Status   | Type               | Library              | License |
+---------------------------------+----------+--------------------+----------------------+---------+
| group_replication               | ACTIVE   | GROUP REPLICATION  | group_replication.so | GPL     |
+---------------------------------+----------+--------------------+----------------------+---------+
46 rows in set (0.04 sec)

In [ ]:
Bootstrapping the Group
==============================================

The process of starting a group for the first time is called bootstrapping.
The bootstrap should only be done by a single server, the one that starts the group and only once.

mysql> SET GLOBAL group_replication_bootstrap_group=ON;
Query OK, 0 rows affected (0.01 sec)

mysql> START GROUP_REPLICATION;
ERROR 3092 (HY000): The server is not configured properly to be an active member of the group. 
Please see more details on error log.
mysql> 

2024-06-12T19:29:22.470271Z 0 [ERROR] [MY-011735] [Repl] Plugin group_replication reported: '[GCS] 
The group communication engine failed to test connectivity t
o the local group communication engine on bosprod01:33061. This may be due to one or more invalid 
configuration settings. Double-check your group replication l
ocal address, firewall, SE Linux and TLS configurations and try restarting Group 
Replication on this server.'


Changing SELinux Mode to Permissive
-----------------------------------------------

[root@bosprod01 ~]# sestatus
SELinux status:                 enabled
SELinuxfs mount:                /sys/fs/selinux
SELinux root directory:         /etc/selinux
Loaded policy name:             targeted
Current mode:                   enforcing
Mode from config file:          enforcing
Policy MLS status:              enabled
Policy deny_unknown status:     allowed
Memory protection checking:     actual (secure)
Max kernel policy version:      33
[root@bosprod01 ~]# 


[root@bosprod01 ~]# vi /etc/selinux/config
[root@bosprod01 ~]# 

SELINUX=permissive

reboot 

[root@bosprod01 ~]# sestatus
SELinux status:                 enabled
SELinuxfs mount:                /sys/fs/selinux
SELinux root directory:         /etc/selinux
Loaded policy name:             targeted
Current mode:                   permissive
Mode from config file:          permissive
Policy MLS status:              enabled
Policy deny_unknown status:     allowed
Memory protection checking:     actual (secure)
Max kernel policy version:      33
[root@bosprod01 ~]# 


mysql> SET GLOBAL group_replication_bootstrap_group=ON;
Query OK, 0 rows affected (0.02 sec)

mysql> START GROUP_REPLICATION;
Query OK, 0 rows affected (6.15 sec)

mysql> SET GLOBAL group_replication_bootstrap_group=OFF;
Query OK, 0 rows affected (0.00 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
1 row in set (0.11 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
1 row in set (0.00 sec)

mysql> create database bosp;
Query OK, 1 row affected (0.03 sec)

mysql> use bosp;
Database changed
mysql> 
mysql> create table orders ( order_id int primary key , order_details varchar(30) , order_date datetime );
Query OK, 0 rows affected (0.14 sec)

mysql> insert into orders values ( 1 , 'gjgdjfdffff', now() );
Query OK, 1 row affected (0.33 sec)

mysql> insert into orders values ( 2 , 'gjgdjfdffff', now() );
Query OK, 1 row affected (0.01 sec)

mysql> insert into orders values ( 3 , 'gjgdjfdffff', now() );
Query OK, 1 row affected (0.00 sec)


In [ ]:
Adding a second instance to the group 
=======================================================

[root@bosprod02 ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

disabled_storage_engines="MyISAM,BLACKHOLE,FEDERATED,ARCHIVE,MEMORY"

server_id=2
gtid_mode=ON
enforce_gtid_consistency=ON

plugin_load_add='group_replication.so'
group_replication_group_name="5842bb1d-28ef-11ef-9a91-000c293ea0bc"
group_replication_start_on_boot=off
group_replication_local_address= "bosprod02:33061"
group_replication_group_seeds= "bosprod01:33061,bosprod02:33061,bosprod03:33061"
group_replication_bootstrap_group=off
[root@bosprod02 ~]# 


[root@bosprod02 ~]# mysqld --validate-config
[root@bosprod02 ~]# 
[root@bosprod02 ~]# systemctl restart mysqld
[root@bosprod02 ~]# 


mysql> set sql_log_bin = 0;
Query OK, 0 rows affected (0.00 sec)

mysql> create user replicator identified by 'Repl@2024';
Query OK, 0 rows affected (0.19 sec)

mysql> grant replication slave on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> grant connection_admin on *.* to replicator;
Query OK, 0 rows affected (0.02 sec)

mysql> grant backup_admin on *.* to replicator;
Query OK, 0 rows affected (0.00 sec)

mysql> grant group_replication_stream on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> flush privileges;
Query OK, 0 rows affected (0.00 sec)

mysql> set sql_log_bin = 1;
Query OK, 0 rows affected (0.00 sec)

mysql> CHANGE REPLICATION SOURCE TO SOURCE_USER='replicator' ,
    -> SOURCE_PASSWORD='Repl@2024'
    -> FOR CHANNEL 'group_replication_recovery';
Query OK, 0 rows affected, 2 warnings (0.09 sec)

mysql> START GROUP_REPLICATION;
ERROR 3092 (HY000): The server is not configured properly to be an active member of the group. 
Please see more details on error log.
mysql> 

[root@bosprod01 ~]# systemctl stop firewalld
[root@bosprod01 ~]# 

[root@bosprod02 ~]# systemctl stop firewalld
[root@bosprod02 ~]# 

[root@bosprod03 ~]# systemctl stop firewalld
[root@bosprod03 ~]# 


mysql> START GROUP_REPLICATION;
Query OK, 0 rows affected (5.91 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
2 rows in set (0.12 sec)

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod02  |        3 |
+------------+----------+
1 row in set (0.17 sec)


mysql> create database posdb;
ERROR 1290 (HY000): The MySQL server is running with the --super-read-only option so it cannot execute this statement
mysql> 
mysql> show variables like '%read_only%';
+-----------------------+-------+
| Variable_name         | Value |
+-----------------------+-------+
| innodb_read_only      | OFF   |
| read_only             | ON    |
| super_read_only       | ON    |
| transaction_read_only | OFF   |
+-----------------------+-------+
4 rows in set (0.19 sec)


In [ ]:
Adding third instance to the group 
=======================================================

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   | 8.0.30         | XCom                       |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
3 rows in set (0.01 sec)
